In [4]:
%%configure
{ 
    "conf": {
        "spark.driver.memory": "4G",
        "spark.jars.packages": "saurfang:spark-sas7bdat:2.1.0-s_2.11", 
        "spark.driver.extraJavaOptions" : "-Dlog4jspark.root.logger=WARN,console" 
  }
}

In [5]:
from urllib.parse import urlparse
import boto3
import logging
import datetime
import pandas as pd

from IPython.display import display, HTML
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql.types import ArrayType, MapType, StringType, DateType, IntegerType, BooleanType, FloatType, LongType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1598776925981_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
pd.set_option("max_columns", 100)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:20.2f}'.format

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# %%local
# import pandas as pd
# pd.set_option("max_columns", 100)
# pd.set_option('display.max_rows', 100)
# pd.options.display.float_format = '{:20.2f}'.format

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# from pyspark.sql import SparkSession
# spark = SparkSession \
#     .builder\
#     .appName("test-packages") \
#     .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,saurfang:spark-sas7bdat:2.1.0-s_2.11")\
#     .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
BUCKET_URL = "s3://udacity-capstone-raw-data"
DATA_PATH = f"{BUCKET_URL}/i94-data"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
SAS_FORMAT_READER = "com.github.saurfang.sas.spark"

def read_sas_data(spark, path: str, columns_to_read=None):

    url_components = urlparse(path)

    if url_components.scheme == 's3':
        s3_client = boto3.client('s3')
        s3_resource = boto3.resource('s3')

        s3_objects = s3_client.list_objects(Bucket=url_components.netloc, Prefix=url_components.path.lstrip("/"))

        df = None
        for key in s3_objects['Contents']:
            file_key = key['Key']
            print("Reading File: {}".format(file_key))
            logging.warning("Reading File: {}".format(file_key))
            file_df = (
                spark.read
                .format(SAS_FORMAT_READER)
                .load("s3://{}/{}".format(url_components.netloc, file_key))
            )
            if columns_to_read:
                file_df = file_df.select(VALID_COLUMNS)
            if df:
                df = df.union(file_df)
            else:
                df = file_df
    else:
        df = (
            spark.read
            .format(SAS_FORMAT_READER)
            .load("s3://{}/{}".format(url_components.netloc, file_key))
        )
        if columns_to_read:
            df.select(columns_to_read)

    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Try to read all data files

In [114]:
read_sas_data(spark, DATA_PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"Union can only be performed on tables with the same number of columns, but the first table has 28 columns and the 6th table has 34 columns;;\n'Union\n:- Relation[cicid#3502,i94yr#3503,i94mon#3504,i94cit#3505,i94res#3506,i94port#3507,arrdate#3508,i94mode#3509,i94addr#3510,depdate#3511,i94bir#3512,i94visa#3513,count#3514,dtadfile#3515,visapost#3516,occup#3517,entdepa#3518,entdepd#3519,entdepu#3520,matflag#3521,biryear#3522,dtaddto#3523,gender#3524,insnum#3525,... 4 more fields] SasRelation(s3://udacity-capstone-raw-data/i94-data/i94_apr16_sub.sas7bdat,null,false,false,false,None,false,false,false,false,60,None,None)\n:- Relation[cicid#3558,i94yr#3559,i94mon#3560,i94cit#3561,i94res#3562,i94port#3563,arrdate#3564,i94mode#3565,i94addr#3566,depdate#3567,i94bir#3568,i94visa#3569,count#3570,dtadfile#3571,visapost#3572,occup#3573,entdepa#3574,entdepd#3575,entdepu#3576,matflag#3577,biryear#3578,dtaddto#3579,gender#3580,insnum#3581,... 4 more fields] SasRelation(s3://ud

One of the files has more columns than the other ones, need to check if these columns matter

In [115]:
regular_df = (
    spark.read
        .format(SAS_FORMAT_READER)
        .load("{}/i94_apr16_sub.sas7bdat".format(DATA_PATH))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
exception_df = (
    spark.read
        .format(SAS_FORMAT_READER)
        .load("{}/i94_jun16_sub.sas7bdat".format(DATA_PATH))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
regular_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype']

In [118]:
exception_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count', 'validres', 'delete_days', 'delete_mexl', 'delete_dup', 'delete_visa', 'delete_recdup', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype']

In [88]:
diff_columns = set(exception_df.columns).difference(regular_df.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
for col in diff_columns:
    print(exception_df.groupBy(col).count().show())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------+
|delete_recdup|  count|
+-------------+-------+
|          0.0|3574989|
+-------------+-------+

None
+-----------+-------+
|delete_mexl|  count|
+-----------+-------+
|        0.0|3574989|
+-----------+-------+

None
+-----------+-------+
|delete_visa|  count|
+-----------+-------+
|        0.0|3574989|
+-----------+-------+

None
+--------+-------+
|validres|  count|
+--------+-------+
|     1.0|3574989|
+--------+-------+

None
+----------+-------+
|delete_dup|  count|
+----------+-------+
|       0.0|3574989|
+----------+-------+

None
+-----------+-------+
|delete_days|  count|
+-----------+-------+
|        0.0|3574989|
+-----------+-------+

None

# Read the full data after removing invalid columns

In [11]:
VALID_COLUMNS = [
    'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate',
    'i94bir', 'i94visa', 'count', 'dtadfile',
    'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear',
    'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df = read_sas_data(spark, DATA_PATH, VALID_COLUMNS)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading File: i94-data/i94_apr16_sub.sas7bdat
Reading File: i94-data/i94_aug16_sub.sas7bdat
Reading File: i94-data/i94_dec16_sub.sas7bdat
Reading File: i94-data/i94_feb16_sub.sas7bdat
Reading File: i94-data/i94_jan16_sub.sas7bdat
Reading File: i94-data/i94_jul16_sub.sas7bdat
Reading File: i94-data/i94_jun16_sub.sas7bdat
Reading File: i94-data/i94_mar16_sub.sas7bdat
Reading File: i94-data/i94_may16_sub.sas7bdat
Reading File: i94-data/i94_nov16_sub.sas7bdat
Reading File: i94-data/i94_oct16_sub.sas7bdat
Reading File: i94-data/i94_sep16_sub.sas7bdat

In [24]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40790529

In [11]:
%%spark -o df_sample
df_sample = df.sample(0.00001).orderBy(F.rand()).limit(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%local
df_sample

Output()

# Exploration

## Source and Destination

### i94port

In [152]:
column = "i94port"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
%%local
column_df

Output()

#### let's take a look at some of the non-US ports of entry, because that doesn't make sense

In [163]:
df[df.i94port.isin(['NAS'])].show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|594.0|2016.0|   4.0| 103.0| 103.0|    NAS|20545.0|    1.0|     FL|20547.0|  27.0|    2.0|  1.0|20160401|    null| null|      G|      N|   null|      M| 1989.0|06292016|     M|  null|     UP|5.5406109033E10|00221|      WT|
|595.0|2016.0|   4.0| 103.0| 103.0|    NAS|20545.0|    1.0|     FL|20547.0|  26.0|    2.0|  1.0|20160401|   

In [171]:
df[df.i94port.isin(['SNO', 'SLP', 'GRU', 'LGW', 'LGW', 'PTY', 'LGW', 'ZZZ', 'MAA'])].show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|  cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|  700.0|2016.0|   4.0| 103.0| 296.0|    MAA|20545.0|    1.0|     CA|20562.0|  36.0|    2.0|  1.0|20160401|     BMB| null|      G|      O|   null|      M| 1980.0|10012016|     F|  null|     EY| 9.249307933E10|00183|      B2|
| 8732.0|2016.0|   4.0| 111.0| 111.0|    MAA|20545.0|    1.0|     IA|20565.0|  28.0|    2.0|  1.0|20

In [172]:
df[df.i94port.isin(['MAA'])].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23441

### i94addr

In [164]:
column = "i94addr"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
%%local
column_df

Output()

In [169]:
%%spark -o null_value_df
null_value_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
%%local
null_value_df

Output()

### i94cit
Citizenship

In [28]:
column = "i94cit"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
%%local
column_df

Output()

In [31]:
%%local
column_df.sum()

i94cit      114811.0
count     40761954.0
dtype: float64

### i94res

In [32]:
column = "i94res"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
%%local
column_df

Output()

In [35]:
%%local
column_df.sum()

i94res       83439.0
count     40790529.0
dtype: float64

### i94res vs i94cit

In [56]:
df_diff = df[df.i94res != df.i94cit].cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
df_diff.createOrReplaceTempView("i94res_i94cit")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df_diff.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6359725

In [60]:
%%sql
SELECT DISTINCT i94cit, i94res
FROM i94res_i94cit
LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## Other Columns

### occup

In [13]:
%%spark -o occup_df
occup_df = df.groupBy('occup').agg(F.count("occup").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%local
occup_df

Output()

In [15]:
%%local
occup_df.sum()

count    192955
dtype: int64

### visatype

In [16]:
column = "visatype"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
%%local
column_df

Output()

In [19]:
%%local
column_df.sum()

visatype    WTB2WBB1F1GMTCPE2F2E1IM1I1GMBM2CPLSBP
count                                    40790529
dtype: object

### i94mode

In [20]:
column = "i94mode"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
%%local
column_df

Output()

In [23]:
%%local
column_df.sum()

i94mode          15.0
count      40716580.0
dtype: float64

### visapost
This is the Department of State where where Visa was issued, don't think we'll use it

In [55]:
column = "visapost"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
%%local
column_df

Output()

In [58]:
%%local
column_df.sum()

count    16758354
dtype: int64

### entdepa
How the passenger was admitted to the US. No idea what the letters mean

In [64]:
column = "entdepa"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
%%local
column_df

Output()

### entdepu
If the traveller's status was updated

In [70]:
column = "entdepu"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
%%local
column_df

Output()

### entdepd
How the traveller departed

In [61]:
column = "entdepd"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
%%local
column_df

Output()

### gender

In [131]:
column = "gender"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
%%local
column_df

Output()

In [77]:
%%local
column_df.sum()

count    36710546
dtype: int64

In [127]:
%%spark -o null_value_df
null_value_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
%%local
null_value_df

Output()

### matflag

In [137]:
column = "matflag"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
%%local
column_df

Output()

In [140]:
%%local
column_df.sum()

count    40790529
dtype: int64

In [121]:
%%spark -o null_value_df
null_value_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
%%local
null_value_df

Output()

## Dates

### arrdate
The range of values looks pretty good, no outliers or invalid values

In [36]:
column = "arrdate"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df.describe(column).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|           arrdate|
+-------+------------------+
|  count|          40790529|
|   mean| 20643.13001496009|
| stddev|100.98698880802502|
|    min|           20454.0|
|    max|           20819.0|
+-------+------------------+

In [38]:
%%spark -o column_df
column_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
%%local
column_df

Output()

### depdate
This one has negative values, doesn't make sense, they need to be replaced with NaNs

In [61]:
column = "depdate"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df.describe(column).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|           depdate|
+-------+------------------+
|  count|          37482517|
|   mean|20659.627880833083|
| stddev|116.65803017360834|
|    min|          -14388.0|
|    max|           48342.0|
+-------+------------------+

In [42]:
%%spark -o column_df
column_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
%%local
column_df

Output()

In [62]:
%%spark -o counts_df
counts_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(column)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
%%local
counts_df

Output()

### dtadfile
Date traveller was added to file. Distribution looks ok

In [45]:
column = "dtadfile"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df.describe(column).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|summary|            dtadfile|
+-------+--------------------+
|  count|            40659479|
|   mean|2.0160697567812737E7|
| stddev|  327.44246148196436|
|    min|            20081124|
|    max|            20170609|
+-------+--------------------+

In [47]:
%%spark -o column_df
column_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
%%local
column_df

Output()

### dtaddto
Date the traveller is admitted to

In [49]:
column = "dtaddto"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df.describe(column).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|           dtaddto|
+-------+------------------+
|  count|          40688978|
|   mean| 6759393.900854463|
| stddev|3592018.9151380444|
|    min|          -00-0000|
|    max|          `1132017|
+-------+------------------+

In [51]:
%%spark -o column_df
column_df = df.withColumn("is_null", F.isnull(column)).groupBy("is_null").agg(F.count("is_null").alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
%%local
column_df

Output()

In [53]:
%%spark -o column_df
column_df = df.groupBy(column).agg(F.count(column).alias("count")).orderBy(F.desc("count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**We Should deal witht the D/S status by splitting it into a separate feature**

In [54]:
%%local
column_df

Output()

# Read the dimension tables

In [13]:
gdp_df = spark.read.option("header", True).format("csv").load(f"{BUCKET_URL}/gdp.csv")
country_ids_df = spark.read.option("header", True).format("csv").load(f"{BUCKET_URL}/country_ids.csv")
states_df = spark.read.option("header", True).format("csv").load(f"{BUCKET_URL}/states.csv")
ports_df = spark.read.option("header", True).format("csv").load(f"{BUCKET_URL}/ports.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
country_ids_df_cleaned = (
    country_ids_df
    .withColumn("country_id", F.col("country_id").cast(IntegerType()))
    .withColumn("country_name", F.initcap(F.col("country_name"))) # title case
    .withColumn("country_name_lower", F.lower(F.col("country_name")))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
gdp_df_cleaned = (
    gdp_df
    .withColumnRenamed("Country Name", "country_name")
    .withColumnRenamed("Country Code", "country_code")
    .withColumnRenamed("Value", "gdp_value")
    .filter(F.col("Year") == "2016")
    .withColumn("gdp_value", F.col("gdp_value").cast(FloatType()).cast(LongType()))
    .withColumn("country_name", F.initcap(F.col("country_name"))) # title case
    .withColumn("country_name_lower", F.lower(F.col("country_name")))
    .drop("Year")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
ports_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|port_id|           port_name|
+-------+--------------------+
|    ALC|           ALCAN, AK|
|    ANC|       ANCHORAGE, AK|
|    BAR|BAKER AAF - BAKER...|
|    DAC|   DALTONS CACHE, AK|
|    PIZ|DEW STATION PT LA...|
|    DTH|    DUTCH HARBOR, AK|
|    EGL|           EAGLE, AK|
|    FRB|       FAIRBANKS, AK|
|    HOM|           HOMER, AK|
|    HYD|           HYDER, AK|
|    JUN|          JUNEAU, AK|
|    5KE|       KETCHIKAN, AK|
|    KET|       KETCHIKAN, AK|
|    MOS|MOSES POINT INTER...|
|    NIK|         NIKISKI, AK|
|    NOM|             NOM, AK|
|    PKC|     POKER CREEK, AK|
|    ORI|  PORT LIONS SPB, AK|
|    SKA|         SKAGWAY, AK|
|    SNP| ST. PAUL ISLAND, AK|
+-------+--------------------+
only showing top 20 rows

In [14]:
gdp_df_cleaned.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+--------------------+
|        country_name|country_code|     gdp_value|  country_name_lower|
+--------------------+------------+--------------+--------------------+
|          Arab World|         ARB| 2504702558208|          arab world|
|Caribbean Small S...|         CSS|   66707361792|caribbean small s...|
|Central Europe An...|         CEB| 1312157728768|central europe an...|
|Early-demographic...|         EAR|10401989263360|early-demographic...|
| East Asia & Pacific|         EAS|22480427155456| east asia & pacific|
|East Asia & Pacif...|         EAP|13512435826688|east asia & pacif...|
|East Asia & Pacif...|         TEA|13486491959296|east asia & pacif...|
|           Euro Area|         EMU|11934055268352|           euro area|
|Europe & Central ...|         ECS|20273841569792|europe & central ...|
|Europe & Central ...|         ECA| 2991823781888|europe & central ...|
|Europe & Central ...|         TEC| 3463188316160|europe & centr

# Cleaning

In [16]:
USEFUL_COLUMNS = [
    "admnum", "i94yr", "i94mon",
    "arrdate", "depdate", "dtaddto",
    "i94cit", "i94res", "i94port", "i94addr",
    "matflag", "i94mode",
    "i94bir", "gender", "visatype", "i94visa",
    "entdepa", "entdepu", "entdepd"
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
clean_df = df.select(USEFUL_COLUMNS)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
clean_df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+------+-------+-------+--------+------+------+-------+-------+-------+-------+------+------+--------+-------+-------+-------+-------+
|       admnum| i94yr|i94mon|arrdate|depdate| dtaddto|i94cit|i94res|i94port|i94addr|matflag|i94mode|i94bir|gender|visatype|i94visa|entdepa|entdepu|entdepd|
+-------------+------+------+-------+-------+--------+------+------+-------+-------+-------+-------+------+------+--------+-------+-------+-------+-------+
|1.897628485E9|2016.0|   4.0|20573.0|   null|10282016| 692.0| 692.0|    XXX|   null|   null|   null|  37.0|  null|      B2|    2.0|      T|      U|   null|
| 3.73679633E9|2016.0|   4.0|20551.0|   null|     D/S| 254.0| 276.0|    ATL|     AL|   null|    1.0|  25.0|     M|      F1|    3.0|      G|      Y|   null|
+-------------+------+------+-------+-------+--------+------+------+-------+-------+-------+-------+------+------+--------+-------+-------+-------+-------+
only showing top 2 rows

In [18]:
@udf(DateType())
def get_date_from_int(x):
    if x is None:
        return x
    if x < 0:
        return datetime.datetime(1900,1,1) # default date for invalid dates
    else:
        return datetime.datetime(1960, 1, 1) + datetime.timedelta(x)

@udf(StringType())
def normalize_gender(x):
    if x in {"M", "F"}:
        return x
    elif x is None:
        return None
    else:
        return "OTHER"

@udf(BooleanType())
def normalize_match_flag(x):
    if x == None:
        return False
    else:
        return True

@udf(StringType())
def normalize_mode(x):
    if x == 1:
        return "Air"
    elif x == 2:
        return "Sea"
    elif x == 3:
        return "Land"
    elif x is None:
        return None
    else:
        return "Other"
    
@udf(StringType())
def normalize_visa_category(x):
    if x == 1:
        return "Business"
    elif x == 2:
        return "Pleasure"
    elif x == 3:
        return "Student"
    elif x is None:
        return None
    else:
        return "Other"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
new_df = (
    clean_df
    .withColumnRenamed('i94yr', 'year')
    .withColumnRenamed('i94mon', 'month')
    .withColumnRenamed('i94res', 'country_residence_id')
    .withColumnRenamed('i94cit', 'country_citizenship_id')
    .withColumnRenamed('i94port', 'entry_port')
    .withColumnRenamed('i94addr', 'destination_state')
    .withColumnRenamed('i94mode', 'travel_mode')
    .withColumnRenamed('i94bir', 'age')
    .withColumnRenamed('i94visa', 'visa_category')
    .withColumnRenamed('visatype', 'visa_type')
    .withColumnRenamed('matflag', 'dates_match_flag')
    .withColumnRenamed('arrdate', 'arrival_date')
    .withColumnRenamed('depdate', 'departure_date')
    .withColumnRenamed('dtaddto', 'permitted_date')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
same_user_window = Window.partitionBy("admnum").orderBy("arrival_date").rowsBetween(Window.unboundedPreceding, -1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
new_df = (
    new_df
    .withColumn('admnum', F.col("admnum").cast(IntegerType()))
    .withColumn('year', F.col("year").cast(IntegerType()))
    .withColumn('month', F.col("month").cast(IntegerType()))
    .withColumn('unrestricted_stay', F.when(F.col('permitted_date') == 'D/S', True).otherwise(False))
    .withColumn('arrival_date', get_date_from_int(F.col("arrival_date")))
    .withColumn('departure_date', get_date_from_int(F.col("departure_date")))
    .withColumn('permitted_date', F.to_date('permitted_date', 'MMddyyyy'))
    .withColumn('age', F.col("age").cast(IntegerType()))
    .withColumn('gender', normalize_gender(F.col("gender")))
    .withColumn('dates_match_flag', normalize_match_flag(F.col("dates_match_flag")))
    .withColumn('travel_mode', normalize_mode(F.col("travel_mode")))
    .withColumn('visa_category', normalize_visa_category(F.col("visa_category").cast(IntegerType())))
    .withColumn('visa_category', normalize_visa_category(F.col("visa_category").cast(IntegerType())))
    .withColumn('country_citizenship_id', F.col("country_citizenship_id").cast(IntegerType()))
    .withColumn('country_residence_id', F.col("country_residence_id").cast(IntegerType()))
    .withColumn('num_previous_stays', F.count("*").over(same_user_window))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
new_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- admnum: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- permitted_date: date (nullable = true)
 |-- country_citizenship: integer (nullable = true)
 |-- country_residence: integer (nullable = true)
 |-- entry_port: string (nullable = true)
 |-- destination_state: string (nullable = true)
 |-- dates_match_flag: boolean (nullable = true)
 |-- travel_mode: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- visa_category: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- unrestricted_stay: boolean (nullable = false)
 |-- num_previous_stays: long (nullable = false)

In [91]:
new_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-----+------------+--------------+--------------+-------------------+-----------------+----------+-----------------+----------------+-----------+---+------+---------+-------------+-------+-------+-------+-----------------+------------------+
|   admnum|year|month|arrival_date|departure_date|permitted_date|country_citizenship|country_residence|entry_port|destination_state|dates_match_flag|travel_mode|age|gender|visa_type|visa_category|entdepa|entdepu|entdepd|unrestricted_stay|num_previous_stays|
+---------+----+-----+------------+--------------+--------------+-------------------+-----------------+----------+-----------------+----------------+-----------+---+------+---------+-------------+-------+-------+-------+-----------------+------------------+
|  1302524|2016|    3|  2016-03-04|          null|    2018-03-03|                691|              691|       MIA|               FL|           false|        Air| 31|     F|       E2|         null|      A|   null|   null|      

In [22]:
joined_df = (
    new_df
    .alias("immigration")
    .join(country_ids_df_cleaned.alias("cit_df"), F.col("immigration.country_citizenship") == F.col("cit_df.country_id"), "leftouter")
    .join(country_ids_df_cleaned.alias("res_df"), F.col("immigration.country_residence") == F.col("res_df.country_id"), "leftouter")
    .join(ports_df.alias("port_df"), F.col("immigration.entry_port") == F.col("port_df.port_id"), "leftouter")
    .selectExpr("immigration.*", "cit_df.country_name as cit_country", "res_df.country_name as res_country", "port_df.port_name")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
joined_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+------------+--------------+--------------+-------------------+-----------------+----------+-----------------+----------------+-----------+---+------+---------+-------------+-------+-----------+-----------+--------------------+
|year|month|arrival_date|departure_date|permitted_date|country_citizenship|country_residence|entry_port|destination_state|dates_match_flag|travel_mode|age|gender|visa_type|visa_category|entdepu|cit_country|res_country|           port_name|
+----+-----+------------+--------------+--------------+-------------------+-----------------+----------+-----------------+----------------+-----------+---+------+---------+-------------+-------+-----------+-----------+--------------------+
|2016|    4|  2016-04-29|          null|    2016-10-28|                692|              692|       XXX|             null|           false|       null| 37|  null|       B2|         null|      U|    Ecuador|    Ecuador|NOT REPORTED/UNKN...|
|2016|    4|  2016-04-07|          null|

We have a lot of invalid citizenship countries

In [156]:
joined_df[~F.isnull(F.col("country_citizenship")) & F.isnull(F.col("cit_country"))].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5017037

We don't have any invalid residence countries

In [157]:
joined_df[~F.isnull(F.col("country_residence")) & F.isnull(F.col("res_country"))].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

We have a relatively small number of invalid ports

In [158]:
joined_df[~F.isnull(F.col("entry_port")) & F.isnull(F.col("port_name"))].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

41128

### Defining the label for our ML Problem

The ML Problem is to figure out if the passenger will overstay his permit. 

Just because your `departure_date` > `permitted_date` doesn't mean that you overstayed

In [37]:
label_df[
    (label_df.dates_match_flag == True) &
    (label_df.unrestricted_stay == False) &
    (label_df.departure_date > label_df.permitted_date) &
    (F.isnull(label_df.entdepu))
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

84717

Let's see what relying on just the `dates_match_flag` will give us

In [23]:
label_df = joined_df.select([
    "admnum", "year", "month", 
    "arrival_date", "departure_date", "permitted_date", "unrestricted_stay", 
    "cit_country", "res_country", 
    "entdepa", "entdepu", "entdepd", 
    "visa_type", "dates_match_flag"
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
label_df = label_df.repartition(8).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
label_df[
    (label_df.dates_match_flag == False)
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3219581

Can the dates match even if there is no departure date? Apparently yes

In [29]:
label_df[
    (label_df.dates_match_flag == True) &
    (F.isnull(label_df.departure_date))
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

88526

In [26]:
label_df[
    (label_df.dates_match_flag == True) &
    (F.isnull(label_df.departure_date))
].show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+-----------+--------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|cit_country|   res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+-----------+--------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    4|  2016-04-29|          null|    2016-07-28|            false|     France|        France|      A|   null|      W|       WT|            true|
|2147483647|2016|    4|  2016-04-26|          null|    2016-05-01|            false|     Sweden|        Sweden|      Z|   null|      W|       WT|            true|
| 672542985|2016|    4|  2016-04-02|          null|    2016-10-01|            false|      Chile|         Chile|      U|   null|      W|       B2|            true|
|2147483647|2016|    4

Let's see if the `entdepd` flag would serve as a surrogate for departing? Yes

In [38]:
label_df[
    (label_df.dates_match_flag == True) &
    (F.isnull(label_df.departure_date)) &
    (F.isnull(label_df.entdepd))
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

This means that for `dates_match_flag` = True, the passenger will always have departed

#### Dates not matching

In [35]:
label_df[
    (label_df.dates_match_flag == False)
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3219581

**Can we trust the match flag = False only?**

##### Unrestricted Stays

Let's see what happens for unrestricted stays (D/S permission dates)

Non-matching unrestricted

In [39]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == True)
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1019714

Only very few of those unrestricted stays who don't match, actually departed

In [42]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == True) &
    (~F.isnull(label_df.departure_date))
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24

In [43]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == True) &
    (~F.isnull(label_df.departure_date))
].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+--------------------+--------------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|         cit_country|         res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+--------------------+--------------------+-------+-------+-------+---------+----------------+
|1638408985|2016|    7|  2016-07-03|    2016-08-27|          null|             true|         Switzerland|         Switzerland|      Z|      U|      O|       F1|           false|
|2147483647|2016|    7|  2016-07-24|    2016-08-13|          null|             true|              Brazil|              Brazil|      G|      U|      Q|       F1|           false|
|2147483647|2016|    7|  2016-07-26|    2016-08-09|          null|             true|                null|     

Matching unrestricted

In [49]:
label_df[
    (label_df.dates_match_flag == True) &
    (label_df.unrestricted_stay == True)
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

573322

How many of those matching unrestricted actually departed (and that's why they are matching)

In [61]:
label_df[
    (label_df.dates_match_flag == True) &
    (label_df.unrestricted_stay == True)
].groupBy(
    (~(F.isnull(F.col("departure_date")))).alias("did_depart")
).count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+
|did_depart| count|
+----------+------+
|      true|573217|
|     false|   105|
+----------+------+

Let's see who those 105 are

Ok, they all have the `entdepd` flag, so they actually departed

In [56]:
label_df[
    (label_df.dates_match_flag == True) &
    (label_df.unrestricted_stay == True) &
    (F.isnull(label_df.departure_date))
].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+--------------+--------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|   cit_country|   res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+--------------+--------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    1|  2016-01-18|          null|          null|             true|     Hong Kong|     Hong Kong|      Z|   null|      W|       F1|            true|
|2147483647|2016|   11|  2016-11-17|          null|          null|             true|         China|         China|      Z|   null|      W|       F1|            true|
|2147483647|2016|   11|  2016-11-27|          null|          null|             true|         China|         China|      Z|   null|      W|       F1|            true|
|214

Let's try again

In [63]:
label_df[
    (label_df.dates_match_flag == True) &
    (label_df.unrestricted_stay == True)
].groupBy(
    (
        (~(F.isnull(F.col("departure_date")))) | 
        (~(F.isnull(F.col("entdepd"))))
    ).alias("did_depart")
).count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+
|did_depart| count|
+----------+------+
|      true|573322|
+----------+------+

So for unrestricted stays, this flag is not reliable, because we don't know if there is no match because the passenger actually overstayed after his Status expired, or if there is no match because he is still in the country

**This means that we for all unrestricted stays, our overstay flag will be = False**

##### Restricted Stays

In [43]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False)
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2199867

In [40]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == True)
].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+------------+------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay| cit_country| res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+------------+------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    4|  2016-04-19|          null|          null|             true|     Nigeria|     Nigeria|      U|   null|   null|       F1|           false|
| 878004585|2016|    4|  2016-04-23|          null|          null|             true|       China|       China|      T|   null|   null|       F1|           false|
|2147483647|2016|    4|  2016-04-21|          null|          null|             true|       Italy|       Italy|      G|   null|   null|        I|           false|
|2147483647|2016|    4|  201

Out of the people who do not match and have restricted stays, very rarely do people actually depart, **those are the ones we are most interested in**, the overstayers

In [68]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    ((~(F.isnull(label_df.departure_date))) | (~(F.isnull(label_df.entdepd))) )
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2413

In [70]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    ((~(F.isnull(label_df.departure_date))) | (~(F.isnull(label_df.entdepd))) )
].show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+--------------+--------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|   cit_country|   res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+--------------+--------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    8|  2016-08-02|          null|    2016-10-30|            false|          null|United Kingdom|      A|   null|      W|       WT|           false|
|2147483647|2016|    8|  2016-08-01|          null|    2016-10-29|            false|          null|United Kingdom|      A|   null|      W|       WT|           false|
|2147483647|2016|    8|  2016-08-03|          null|    2016-10-31|            false|          null|United Kingdom|      A|   null|      W|       WT|           false|
|214

Most of them have a W entdepd, which is a waiver because their visa is usually the WT visa, so they probably got an extension somehow and shouldn't be considered overstayers

In [71]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    ((~(F.isnull(label_df.departure_date))) | (~(F.isnull(label_df.entdepd))) )
].groupBy(label_df.entdepd).count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
|entdepd|count|
+-------+-----+
|      D|    3|
|      R|    6|
|      J|    3|
|      N|   19|
|      V|   40|
|      O|   23|
|      W| 2253|
|      I|   56|
|      K|   10|
+-------+-----+

In [72]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    (((F.isnull(label_df.departure_date))) & ((F.isnull(label_df.entdepd))) )
].count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2197454

Last Detail is: Let's see how many of those overstayers updated their visas

In [76]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    (((F.isnull(label_df.departure_date))) & ((F.isnull(label_df.entdepd))) )
].groupBy("entdepu").agg(F.count("*").alias("count")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+
|entdepu|  count|
+-------+-------+
|   null|2195077|
|      U|   2375|
|      P|      2|
+-------+-------+

In [78]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    (((F.isnull(label_df.departure_date))) & ((F.isnull(label_df.entdepd))) ) &
    (~F.isnull(label_df.entdepu))
].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+--------------------+--------------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|         cit_country|         res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+--------------------+--------------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    4|  2016-04-23|          null|    2018-04-22|            false|               Japan|               Japan|      G|      U|   null|       E2|           false|
|2147483647|2016|    4|  2016-04-20|          null|    2018-04-19|            false|                null|         South Korea|      O|      U|   null|       E2|           false|
|2147483647|2016|    4|  2016-04-28|          null|    2016-10-27|            false|               India|     

There aren't a lot of them, but let's assume they upgraded to a permanent visa, so let's exclude them too

In [79]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    (((F.isnull(label_df.departure_date))) & ((F.isnull(label_df.entdepd))) ) &
    (F.isnull(label_df.entdepu))
].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-----+------------+--------------+--------------+-----------------+------------------+------------------+-------+-------+-------+---------+----------------+
|    admnum|year|month|arrival_date|departure_date|permitted_date|unrestricted_stay|       cit_country|       res_country|entdepa|entdepu|entdepd|visa_type|dates_match_flag|
+----------+----+-----+------------+--------------+--------------+-----------------+------------------+------------------+-------+-------+-------+---------+----------------+
|2147483647|2016|    4|  2016-04-20|          null|    2016-10-19|            false|           Ukraine|           Ukraine|      G|   null|   null|       B2|           false|
|2147483647|2016|    4|  2016-04-23|          null|    2016-10-22|            false|          Pakistan|          Pakistan|      O|   null|   null|       B2|           false|
|2147483647|2016|    4|  2016-04-22|          null|    2016-07-20|            false|            Monaco|       Netherlands|      G|

### Creating the label

In [81]:
joined_df = joined_df.withColumn(
    "is_overstay", 
    F.when(
        (
            (joined_df.dates_match_flag == False) &
            (joined_df.unrestricted_stay == False) &
            ((F.isnull(label_df.departure_date)) & ((F.isnull(label_df.entdepd)))) &
            (F.isnull(label_df.entdepu))
        ),
        True
    ).otherwise(False)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
joined_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- admnum: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- permitted_date: date (nullable = true)
 |-- country_citizenship: integer (nullable = true)
 |-- country_residence: integer (nullable = true)
 |-- entry_port: string (nullable = true)
 |-- destination_state: string (nullable = true)
 |-- dates_match_flag: boolean (nullable = true)
 |-- travel_mode: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- visa_category: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- unrestricted_stay: boolean (nullable = false)
 |-- num_previous_stays: long (nullable = false)
 |-- cit_country: string (nullable = true)
 |-- res_country: string (nullable = true)
 |-

In [ ]:
FINAL_COLUMNS = [
    "year", "month", 
    "arrival_date", "departure_date", "permitted_date", "unrestricted_stay",
    "country_citizenship", "country_residence", "port_name", "destination_state", 
    "age", "gender", "num_previous_stays", 
    "visa_type", "visa_category"
    "is_overstay"
]

In [ ]:
label_df[
    (label_df.dates_match_flag == False) &
    (label_df.unrestricted_stay == False) &
    (((F.isnull(label_df.departure_date))) & ((F.isnull(label_df.entdepd))) ) &
    (F.isnull(label_df.entdepu))
].show()

# Backup

In [17]:
%%logs

stdout: 

stderr: 
20/08/23 12:49:18 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 45981.
20/08/23 12:49:18 INFO NettyBlockTransferService: Server created on ip-10-0-1-139.us-west-2.compute.internal:45981
20/08/23 12:49:18 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy
20/08/23 12:49:18 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, ip-10-0-1-139.us-west-2.compute.internal, 45981, None)
20/08/23 12:49:18 INFO BlockManagerMasterEndpoint: Registering block manager ip-10-0-1-139.us-west-2.compute.internal:45981 with 912.3 MB RAM, BlockManagerId(driver, ip-10-0-1-139.us-west-2.compute.internal, 45981, None)
20/08/23 12:49:18 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, ip-10-0-1-139.us-west-2.compute.internal, 45981, None)
20/08/23 12:49:18 INFO BlockManager: external shuffle service port = 7337
20/08/23 12:49:18 INF

In [12]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1598201641626_0001,pyspark,idle,Link,Link,
2,application_1598201641626_0003,pyspark,idle,Link,Link,✔


In [11]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.
send_to_spark,%%send_to_spark -o variable -t str -n var,"Sends a variable from local output to spark cluster. Parameters: -i VAR_NAME: Local Pandas DataFrame(or String) of name VAR_NAME will be available in the %%spark context as a Spark dataframe(or String) with the same name. -t TYPE: Specifies the type of variable passed as -i. Available options are: `str` for string and `df` for Pandas DataFrame. Optional, defaults to `str`. -n NAME: Custom name of variable passed as -i. Optional, defaults to -i variable name. -m MAXROWS: Maximum amount of Pandas rows that will be sent to Spark. Defaults to 2500."
